# Segmenting and Clustering Neighborhoods in Toronto

I made only one notebook for all the assignments.

### Libraries

In [1]:
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import folium

####  Get data from wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

table = pd.read_html(link)
table = table[0].iloc[:]

table.shape

(180, 3)

In [3]:
table.columns=["PostalCode","Borough","Neighbourhood"]
table.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Check how many rows have the same postal code

In [4]:
counter=0
            
for num, l in enumerate(table.PostalCode[:-1]):
    for l2 in table.PostalCode[num+1:]:
        if l==l2: counter+=1
            
print("number of rows having the same Postal Code: {}".format(counter))

number of rows having the same Postal Code: 0


#### Drop "Not assigned" values in Borough

In [5]:
for index, row in table[:].iterrows():
    if row[1]=="Not assigned":
        table.drop(index,axis=0, inplace=True)

table.reset_index(drop=True)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
table.shape

(103, 3)

#### Check how many rows have "Not assigned" neighbourhoods

In [7]:
number_not_assigned=0

for index, row in table[:].iterrows():
        if row[2]=="Not assigned":
            number_not_assigned+=1
            
print("number of not assigned neighbourhoods: {}".format(number_not_assigned))


number of not assigned neighbourhoods: 0


#### Print shape and the first five rows

In [8]:
print(table.shape)
table.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Get latitude and longitude for each postal code 

geocoder is absolutely unreliable. For this reason I am going to use the csv file

In [9]:
# read the csv file
df_pc=pd.read_csv("http://cocl.us/Geospatial_data")

# change the columns' names
df_pc.columns=["PostalCode","Latitude", "Longitude"]

#print shape and head()
print(df_pc.shape)
df_pc.head()

(103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Assign the coordinates to each postal code

In [10]:
latitude=[]
longitude=[]

for postal_code in table.PostalCode:
        for index, row in df_pc.iterrows():
            if(postal_code==row[0]):
                latitude.append(row[1])
                longitude.append(row[2])
                
table["Latitude"]=latitude
table["Longitude"]=longitude

print(table.shape)
table.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Plot the map of the neighbourhoods

In [11]:
tmap=folium.Map(location=[43.651070, -79.347015], zoom_start=11)

locat=folium.map.FeatureGroup()

for lat,lon in zip(latitude, longitude):
    
    locat.add_child(
    folium.vector_layers.CircleMarker([lat,lon], radius=5, color="red", fill_color="red")  
    )
    
tmap.add_child(locat)
tmap

#### Restrict the analysis to the boroughs containing "Toronto"

In [12]:
df_tor = table[table['Borough'].str.contains("Toronto") ].reset_index(drop=True)
print(df_tor.shape)
df_tor.head()

(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Plot only these neighbourhoods

In [13]:
tormap=folium.Map(location=[43.651070, -79.347015], zoom_start=12)

locat=folium.map.FeatureGroup()

for lat,lon in zip(df_tor["Latitude"], df_tor["Longitude"]):
    
    locat.add_child(
    folium.vector_layers.CircleMarker([lat,lon], radius=5, color="red", fill_color="red")  
    )
    
tormap.add_child(locat)
tormap

In [14]:
# @hidden_cell

CLIENT_ID = 'BQ540JB1FDS2SYUG50DXIKUHXQFDU5JZIXD4LTGWOJUBYIGP' # your Foursquare ID
CLIENT_SECRET = 'OKHOCS2SWQFYMDLBA1AJRO2M5IWOALRN2OGJEM3SC2JMN3DP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BQ540JB1FDS2SYUG50DXIKUHXQFDU5JZIXD4LTGWOJUBYIGP
CLIENT_SECRET:OKHOCS2SWQFYMDLBA1AJRO2M5IWOALRN2OGJEM3SC2JMN3DP


#### Define limit and radius to get venues associated to each location

In [15]:
LIMIT=100
radius=500

#### Define the function to obtain venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Call the function and collect the venues in toronto_venues 

In [17]:
toronto_venues = getNearbyVenues(names=df_tor['Neighbourhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Print the shape and head() of toronto_venues

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1647, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Count the number of venues retrieved for each nieighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


#### One hot encoding

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1647, 234)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group by neighbourhood

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(39, 234)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.080000,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.052632,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.014925,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014925,0.0,0.0,0.014925,0.0,0.0


#### Return the top 5 venues of each neighbourhood

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.11
1    Cocktail Bar  0.04
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Yoga Studio  0.08
2          Bakery  0.08
3  Breakfast Spot  0.08
4     Coffee Shop  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Auto Workshop  0.05
3             Brewery  0.05
4          Smoke Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3                Plane  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0

#### Get the top 10 venues associated to each neighbourhood for clustering

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Seafood Restaurant,Bakery,Beer Bar,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Plane,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Department Store,Bubble Tea Shop,Burger Joint,Yoga Studio


#### Apply k-means clustering (k=5) and obtain the labels

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Add clustering labels

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Dessert Shop,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Gym,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Electronics Store,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


#### Plot the map of the neighbourhoods using different colors for different clusters

In [27]:
tormap_cluster=folium.Map(location=[43.651070, -79.347015], zoom_start=11)
color_list=["red", "yellow", "green", "blue", "orange"]

locat=folium.map.FeatureGroup()

for lat, lon, cluster_label in zip(toronto_merged["Latitude"], toronto_merged["Longitude"], toronto_merged["Cluster Labels"]):
    
    locat.add_child(
    folium.vector_layers.CircleMarker([lat,lon], radius=5, color=color_list[cluster_label], fill_color=color_list[cluster_label])  
    )
    
tormap_cluster.add_child(locat)
tormap_cluster

#### Analyze the clusters

The first cluster mainly contains neighborhoods located in the downtown. Coffee shops are the most common venues alognside with café and restaurants.

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Dessert Shop,Shoe Store,Restaurant
1,Downtown Toronto,0,Coffee Shop,Diner,Gym,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Electronics Store,Pizza Place
3,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant
5,Downtown Toronto,0,Coffee Shop,Café,Seafood Restaurant,Bakery,Beer Bar,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Liquor Store
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Department Store,Bubble Tea Shop,Burger Joint,Yoga Studio
7,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Coffee Shop
8,Downtown Toronto,0,Coffee Shop,Café,Gym,Bar,Hotel,Restaurant,Clothing Store,Steakhouse,Thai Restaurant,Sushi Restaurant
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Music Venue,Bar
13,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Concert Hall


The second cluster contains places mainly (but not only) located on the coast. Venues include bars, cafés, and ethnic restaurants.

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,1,Pub,Health Food Store,Trail,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
9,West Toronto,1,Bakery,Pharmacy,Pizza Place,Bank,Bar,Café,Supermarket,Music Venue,Middle Eastern Restaurant,Park
11,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Men's Store,Vietnamese Restaurant,Restaurant,Café,Yoga Studio,Brewery,Record Shop
12,East Toronto,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Furniture / Home Store,Indian Restaurant,Spa,Pub,Lounge
14,West Toronto,1,Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant
15,East Toronto,1,Park,Sushi Restaurant,Board Shop,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pizza Place,Pet Store
22,West Toronto,1,Mexican Restaurant,Café,Thai Restaurant,Discount Store,Speakeasy,Bar,Diner,Bakery,Cajun / Creole Restaurant,Fried Chicken Joint
25,West Toronto,1,Gift Shop,Breakfast Spot,Dessert Shop,Dog Run,Restaurant,Coffee Shop,Eastern European Restaurant,Bookstore,Bar,Cuban Restaurant
27,Downtown Toronto,1,Café,Restaurant,Bar,Bookstore,Japanese Restaurant,Sandwich Place,Bakery,Pub,Dessert Shop,Sushi Restaurant
30,Downtown Toronto,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bar,Bakery,Gaming Cafe,Park,Burger Joint


The third cluster includes places located near parks, trails, and restaurants. It includes places located in the upper part of the city.

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
29,Central Toronto,2,Park,Restaurant,Trail,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
33,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


The fourth cluster is similar to the third one. However, no trails are in the neighbourhoods.

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Dim Sum Restaurant,Swim School,Bus Line,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


The last cluster is very different from the others and it contains the farthest place from the city centre. It is possible to find home services and stores in the nearbies.

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,Garden,Music Venue,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
